# Goal:
- Read a .pkl dataset of proteins
- Dynamically create PostgreSQL table
- Insert proteins + PDBs while preserving order
- Export back to .pkl exactly as input
- Fully dynamic: no manual column specification

# Prerequisites
- **postgreSQL** - Download and install it on your system
- Download_Link -> https://www.postgresql.org/download/windows/
- during installation - create and save superuser, pwd - which you need latter
- **Python** > 3.9+

In [1]:
# =========================================
# STEP 0: Imports
# =========================================
import pickle
import json
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text

# ---------------- CONFIG ---------------- #
PKL_PATH = "../data/ghazi/ENST00000684719_analyzed_protein.pkl"
OUTPUT_PKL = "../data/ghazi/ENST00000684719_analyzed_protein_all_dynamic5.pkl"

PG_USER = "postgres"
PG_PASSWORD = "manjoor123$ps"
PG_HOST = "localhost"
PG_PORT = "5432"
DB_NAME = "protein_db_dynamic5"

TABLE_NAME = "protein_table_dynamic5"

In [2]:
# =========================================
# STEP 1: Load dataset
# =========================================
with open(PKL_PATH, "rb") as f:
    data = pickle.load(f)

print("Total proteins:", len(data))
print("PDB files per protein:", [len(e.get("pdb_files", [])) for e in data])

# Detect columns dynamically from first protein
first_entry = data[0]
pdb_columns = ["pdb_ids", "pdb_files"]
other_columns = [k for k in first_entry.keys() if k != "pdb_files"]
columns = ["protein_index"] + other_columns + pdb_columns
print("Detected columns:", columns)

Total proteins: 1
PDB files per protein: [130]
Detected columns: ['protein_index', 'gene_id', 'transcript_id', 'sequence', 'exons', 'protein_coding', 'nmd', 'analysis_data', 'pdb_ids', 'pdb_files']


In [3]:
# =========================================
# STEP 2: Connect to PostgreSQL (create DB if needed)
# =========================================

# Admin connection to create DB
admin_engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/postgres",
    isolation_level="AUTOCOMMIT"
)

with admin_engine.connect() as conn:
    result = conn.execute(
        text("SELECT 1 FROM pg_database WHERE datname=:name"), {"name": DB_NAME}
    ).fetchone()
    if not result:
        conn.execute(text(f"CREATE DATABASE {DB_NAME}"))
        print(f"Database '{DB_NAME}' created.")
    else:
        print(f"Database '{DB_NAME}' already exists.")

# Connect to the target database
engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{DB_NAME}"
)

Database 'protein_db_dynamic5' created.


In [4]:
# =========================================
# STEP 3: Create table dynamically
# =========================================

# Map Python types to PostgreSQL types
def infer_pg_type(key, value):
    if key == "protein_index":
        return "INTEGER NOT NULL"
    elif key == "pdb_ids":
        return "TEXT[]"
    elif key == "pdb_files":
        return "BYTEA[]"
    elif isinstance(value, bool):
        return "BOOLEAN"
    elif isinstance(value, (dict, list)):
        return "JSONB"
    else:
        return "VARCHAR(400)"


# Build column definitions
col_defs = []
for col in columns:
    pg_type = infer_pg_type(col, first_entry.get(col, None))
    not_null = "NOT NULL" if col in ["gene_id", "transcript_id", "protein_index"] else ""
    col_defs.append(f"{col} {pg_type} {not_null}")

col_defs_sql = ",\n".join(col_defs)

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
{col_defs_sql},
PRIMARY KEY (gene_id, transcript_id)
);
"""

with engine.begin() as conn:
    conn.execute(text(create_table_sql))

print(f"Table '{TABLE_NAME}' created or already exists.")

Table 'protein_table_dynamic5' created or already exists.


In [5]:
# =========================================
# STEP 4: Insert proteins dynamically
# =========================================
def insert_proteins_with_pdbs(engine, data, table_name):
    """
    Insert proteins with PDB files into the database.
    Preserves original PKL order using protein_index.
    """

    # Build INSERT query dynamically
    cols_sql = ", ".join(columns)
    vals_sql = ", ".join([f"%({c})s" for c in columns])
    sql = f"""
    INSERT INTO {table_name} ({cols_sql})
    VALUES ({vals_sql})
    ON CONFLICT (gene_id, transcript_id) DO NOTHING;
    """

    conn = engine.raw_connection()
    try:
        cur = conn.cursor()
        total_pdbs = 0

        for idx, entry in enumerate(data):
            row_dict = {"protein_index": idx}

            # Add all other columns dynamically
            for key in other_columns:
                if key == "exons":
                    row_dict[key] = json.dumps(entry.get(key, []))
                else:
                    row_dict[key] = entry.get(key)

            # PDB fields
            row_dict["pdb_ids"] = [p["pdb_id"] for p in entry.get("pdb_files", [])]
            row_dict["pdb_files"] = [psycopg2.Binary(p["content"]) for p in entry.get("pdb_files", [])]

            total_pdbs += len(row_dict["pdb_files"])

            cur.execute(sql, row_dict)

        conn.commit()

    finally:
        conn.close()

    print(f"✅ Inserted {len(data)} proteins with {total_pdbs} total PDB files.")


# Run insert
insert_proteins_with_pdbs(engine, data, TABLE_NAME)

ProgrammingError: can't adapt type 'dict'

In [9]:
# =========================================
# STEP 5: Verify table content (optional preview)
# =========================================
df_preview = pd.read_sql(f"""
SELECT gene_id, transcript_id, cardinality(pdb_files) AS pdb_count
FROM {TABLE_NAME}
ORDER BY protein_index;
""", engine)
df_preview.head()

,gene_id,transcript_id,pdb_count
0,ENSG00000188938,ENST00000649557,810
1,ENSG00000188938,ENST00000375412,396
2,ENSG00000188938,ENST00000423591,779
3,ENSG00000188938,ENST00000476484,864
4,ENSG00000188938,ENST00000428378,802


In [10]:
# =========================================
# STEP 6: Export full table dynamically
# =========================================
def export_table_to_pkl(engine, table_name, output_path):
    """
    Export entire table back to a .pkl file dynamically.
    Preserves original PKL order using protein_index.
    """

    df = pd.read_sql(f"SELECT * FROM {table_name} ORDER BY protein_index", engine)
    data_out = []

    for _, row in df.iterrows():
        assert len(row["pdb_ids"] or []) == len(row["pdb_files"] or [])

        protein_entry = {}
        for col in other_columns:
            if col == "exons":
                protein_entry[col] = row[col] or []
            else:
                protein_entry[col] = row[col]

        protein_entry["pdb_files"] = [
            {"pdb_id": pid, "content": bytes(pb)} for pid, pb in zip(row["pdb_ids"] or [], row["pdb_files"] or [])
        ]

        data_out.append(protein_entry)

    with open(output_path, "wb") as f:
        pickle.dump(data_out, f)

    print(f"✅ Exported {len(data_out)} proteins (order-preserved and identity-safe)")


# Example usage
export_table_to_pkl(engine, TABLE_NAME, OUTPUT_PKL)

✅ Exported 5 proteins (order-preserved and identity-safe)


In [11]:
# =========================================
# STEP 7: Proof comparator (identity + order)
# =========================================
import hashlib

def hash_protein(protein):
    h = hashlib.sha256()
    h.update(protein["gene_id"].encode())
    h.update(protein["transcript_id"].encode())
    h.update((protein["sequence"] or "").encode())
    h.update(pickle.dumps(protein.get("exons", [])))
    h.update(str(protein.get("protein_coding", False)).encode())
    h.update(str(protein.get("nmd", False)).encode())
    for pdb in protein.get("pdb_files", []):
        h.update(pdb["pdb_id"].encode())
        h.update(pdb["content"])
    return h.hexdigest()


def proof_compare_pkl(file1, file2):
    with open(file1, "rb") as f1, open(file2, "rb") as f2:
        data1, data2 = pickle.load(f1), pickle.load(f2)

    if len(data1) != len(data2):
        print(f"❌ Different number of proteins: {len(data1)} vs {len(data2)}")
        return False

    for i, (p1, p2) in enumerate(zip(data1, data2)):
        if hash_protein(p1) != hash_protein(p2):
            print(f"❌ Mismatch at protein index {i}")
            return False

    print("✅ PROOF PASSED: PKL files are identical (order + content)")
    return True


# Run proof
proof_compare_pkl(PKL_PATH, OUTPUT_PKL)

✅ PROOF PASSED: PKL files are identical (order + content)


True

# END OF NOTE BOOK